In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter.ttk import Combobox
import mysql.connector
from datetime import date
from tkcalendar import DateEntry

#  Database Connection 
def connect_db():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='mayu',
        database='hospital'
    )

#  Add Appointment 
def add_appointment():
    name = entry_name.get()
    age = entry_age.get()
    gender = gender_var.get()
    contact = entry_contact.get()
    doctor = entry_doctor.get()
    date_str = cal.get_date().strftime("%Y-%m-%d")  # YYYY-MM-DD

    if name == "" or age == "" or gender == "" or doctor == "":
        messagebox.showerror("Error", "Please fill all required fields")
        return

    if not age.isdigit() or not contact.isdigit():
        messagebox.showerror("Error", "Age and Contact must be numbers")
        return

    conn = connect_db()
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO appointments (patient_name, age, gender, contact, doctor, appointment_date)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (name, age, gender, contact, doctor, date_str))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Appointment booked successfully!")
    clear_fields()
   

# View Appointments 
def view_appointments():
    conn = connect_db()
    cur = conn.cursor()
    cur.execute("SELECT * FROM appointments")
    rows = cur.fetchall()
    listbox_appointments.delete(0, tk.END)
    
    for i, row in enumerate(rows):
        display_text = f"{row[0]:<3} | {row[1]:<15} | {row[2]:<3} | {row[3]:<10} | {row[4]:<15} | {row[5]:<15} | {row[6]}"
        listbox_appointments.insert(tk.END, display_text)
        # Alternate row colors
        if i % 2 == 0:
            listbox_appointments.itemconfig(i, bg="#e6f2ff")  # light blue
        else:
            listbox_appointments.itemconfig(i, bg="white")
    conn.close()

#  Clear Fields 
def clear_fields():
    entry_name.delete(0, tk.END)
    entry_age.delete(0, tk.END)
    gender_var.set("")
    entry_contact.delete(0, tk.END)
    entry_doctor.set("")
    cal.set_date(date.today())

#  Delete Appointment 
def delete_appointment():
    selected = listbox_appointments.get(tk.ACTIVE)
    if not selected:
        messagebox.showerror("Error", "Select an appointment to delete")
        return
    appointment_id = selected.split(" | ")[0].strip()
    conn = connect_db()
    cur = conn.cursor()
    cur.execute("DELETE FROM appointments WHERE id=%s", (appointment_id,))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Appointment deleted successfully!")
    view_appointments()

#  Hover Effects for Buttons -
def on_enter(e):
    e.widget['bg'] = '#5cb85c'  # light green

def on_leave(e):
    e.widget['bg'] = '#32cd32'  # original green

# Main Window 
win = tk.Tk()
win.title("🏥 Hospital Appointment Booking System")
win.geometry("950x600")
win.configure(bg="#f0f8ff")  # Light blue background

# Title 
tk.Label(
    win, text="Hospital Appointment Booking System",
    font=("Helvetica", 18, "bold"),
    bg="#4682b4", fg="white", pady=10
).pack(fill="x")

# Frames 
form_frame = tk.Frame(win, bg="#f0f8ff")
form_frame.pack(pady=10)

button_frame = tk.Frame(win, bg="#f0f8ff")
button_frame.pack(pady=10)

list_frame = tk.Frame(win, bg="#f0f8ff")
list_frame.pack(pady=10)

# Form Fields 
label_font = ("Arial", 12, "bold")
entry_font = ("Arial", 11)

# Patient Name
tk.Label(form_frame, text="Patient Name:", bg="#f0f8ff", font=label_font).grid(row=0, column=0, padx=10, pady=5, sticky="w")
entry_name = tk.Entry(form_frame, font=entry_font, width=25)
entry_name.grid(row=0, column=1, padx=10, pady=5)

# Age
tk.Label(form_frame, text="Age:", bg="#f0f8ff", font=label_font).grid(row=1, column=0, padx=10, pady=5, sticky="w")
entry_age = tk.Entry(form_frame, font=entry_font, width=25)
entry_age.grid(row=1, column=1, padx=10, pady=5)

# Gender
tk.Label(form_frame, text="Gender:", bg="#f0f8ff", font=label_font).grid(row=2, column=0, padx=10, pady=5, sticky="w")
gender_var = tk.StringVar()
tk.Radiobutton(form_frame, text="Male", variable=gender_var, value="Male", bg="#f0f8ff", font=("Arial", 10)).grid(row=2, column=1, sticky="w")
tk.Radiobutton(form_frame, text="Female", variable=gender_var, value="Female", bg="#f0f8ff", font=("Arial", 10)).grid(row=2, column=1, padx=80, sticky="w")

# Contact
tk.Label(form_frame, text="Contact:", bg="#f0f8ff", font=label_font).grid(row=3, column=0, padx=10, pady=5, sticky="w")
entry_contact = tk.Entry(form_frame, font=entry_font, width=25)
entry_contact.grid(row=3, column=1, padx=10, pady=5)

# Doctor (Combobox)
tk.Label(form_frame, text="Doctor:", bg="#f0f8ff", font=label_font).grid(row=4, column=0, padx=10, pady=5, sticky="w")
entry_doctor = Combobox(form_frame, values=["Dr.Reena(gynicologist)","Dr.Radhi(MD)","Dr.Arjun(orthopedic)","Dr:Teju(pediatrician)","Dr:Naveen(Intern)"], font=entry_font, width=23)
entry_doctor.grid(row=4, column=1, padx=10, pady=5)

# Appointment Date
tk.Label(form_frame, text="Appointment Date:", bg="#f0f8ff", font=label_font).grid(row=5, column=0, padx=10, pady=5, sticky="w")
cal = DateEntry(form_frame, width=22, background='#4682b4', foreground='white', borderwidth=2)
cal.grid(row=5, column=1, padx=10, pady=5)

#  Buttons 
btn_style = {"font": ("Arial", 11, "bold"), "width": 18, "pady": 5}

btn_book = tk.Button(button_frame, text="Book Appointment", command=add_appointment, bg="#32cd32", fg="white", **btn_style)
btn_book.grid(row=0, column=0, padx=10)
btn_book.bind("<Enter>", on_enter)
btn_book.bind("<Leave>", on_leave)

tk.Button(button_frame, text="View Appointments", command=view_appointments, bg="#4682b4", fg="white", **btn_style).grid(row=0, column=1, padx=10)
tk.Button(button_frame, text="Clear Fields", command=clear_fields, bg="#ffa500", fg="white", **btn_style).grid(row=0, column=2, padx=10)
tk.Button(button_frame, text="Delete Appointment", command=delete_appointment, bg="#dc143c", fg="white", **btn_style).grid(row=0, column=3, padx=10)

#  Listbox 
tk.Label(list_frame, text="📋 All Appointments", bg="#f0f8ff", font=("Helvetica", 14, "bold")).pack(anchor="w", padx=10)

scroll = tk.Scrollbar(list_frame)
scroll.pack(side=tk.RIGHT, fill=tk.Y)

listbox_appointments = tk.Listbox(
    list_frame, width=120, height=12, font=("Courier New", 10),
    yscrollcommand=scroll.set, selectbackground="#4682b4", selectforeground="white"
)
listbox_appointments.pack(pady=10)
scroll.config(command=listbox_appointments.yview)

win.mainloop()
